In [1]:
#pip install todoist-python
from todoist import TodoistAPI, models
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
credentials = dotenv_values('../../usage/.env')
path_data = "../../usage/data/"

In [3]:
api = TodoistAPI(credentials["TOKEN"])
s = api.sync()

In [4]:
def get_todoist_activities(api, pages, limit=100):
    activities = pd.DataFrame()
    for p in range(pages):
        if p > 0:
            activities = all_activities
        activities_on_page_df = pd.DataFrame(api.activity.get(page=p, limit=100)['events'])                                                                       
        all_activities = pd.concat([activities, activities_on_page_df])
    return all_activities

def get_parent_id(api, object_id):
    object_id = str(object_id)
    info_list = [api.items.get_by_id(object_id)] 
    for item in info_list:
        try:
            if ('item' in item.keys()) and ('parent_id' in item['item'].keys()):
                return item['item']['parent_id']
        except AttributeError:
            return None

In [5]:
df = get_todoist_activities(api, pages=5, limit=100)

In [6]:
df['content'] = df['extra_data'].apply(lambda x: x['content'] if 'content' in x.keys() else None)
df['due_date'] = df['extra_data'].apply(lambda x: x['due_date'] if 'due_date' in x.keys() else None)
df['name'] = df['extra_data'].apply(lambda x: x['name'] if 'name' in x.keys() else None)
object_id_list = [get_parent_id(api, idd) for idd in df['object_id'].unique()]
parent_id_dp = pd.DataFrame([object_id_list, df['object_id'].unique()]).T
parent_id_dp = parent_id_dp.rename(columns={0:'parent_id', 1: 'object_id'}).astype(int, errors='ignore')
df = df.merge(parent_id_dp, on='object_id')

In [146]:
df['event_date'].min(), df['event_date'].max()

('2022-03-14T12:25:37Z', '2022-04-14T23:05:58Z')

In [7]:
df['event_date'].min(), df['event_date'].max()

('2022-03-23T16:17:08Z', '2022-04-19T12:49:38Z')

In [8]:
df.head(5)

,event_date,event_type,extra_data,id,initiator_id,object_id,object_type,parent_item_id,parent_project_id,content,due_date,name,parent_id
0,2022-04-19T12:49:38Z,completed,"{'client': 'Mozilla/5.0; Todoist/1998', 'conte...",15669775765,NaN,5711662561,item,NaN,2288032544,Fazer exercícios (1),None,None,NaN
1,2022-04-19T03:12:19Z,updated,"{'client': 'Todoist-Android/v9520', 'content':...",15663958368,NaN,5711662561,item,NaN,2288032544,Fazer exercícios (1),2022-04-20T02:59:59Z,None,NaN
2,2022-04-18T12:15:13Z,updated,"{'client': 'Todoist-Android/v9520', 'content':...",15655204747,NaN,5711662561,item,NaN,2288032544,Fazer exercícios (1),2022-04-19T02:59:59Z,None,NaN
3,2022-04-16T10:39:13Z,updated,"{'client': 'Todoist-Android/v9498', 'content':...",15636570150,NaN,5711662561,item,NaN,2288032544,Fazer exercícios (1),2022-04-17T02:59:59Z,None,NaN
4,2022-04-15T01:20:28Z,updated,"{'client': 'Mozilla/5.0; Todoist/1998', 'conte...",15622673622,NaN,5711662561,item,NaN,2288032544,Fazer exercícios (1),2022-04-16T02:59:59Z,None,NaN


In [9]:
# Pega o arquivo com name enddate mais rescente
from glob import glob
import os
from datetime import date, datetime, timedelta
glob_search = f"{path_data}*.csv"

max_dates_on_folder = [
    d.split("/")[-1].split('-')[-1].split('.')[0] for d in glob(glob_search)
]
def get_min_diff_date_from_today(list_of_dates):
    diff_dates = {
        datetime.today() - datetime.strptime(d,"%Y%m%d" ):d 
        for d in list_of_dates
    }
    min_diff = min(list(diff_dates.keys()))
    return diff_dates[min_diff]
    
current_state_date = get_min_diff_date_from_today(max_dates_on_folder)

# filtra os registros que derem match com a data

df['event_date_formated'] = pd.to_datetime(
    df['event_date'], infer_datetime_format=True
).dt.strftime("%Y%m%d" )
df_new = df[df['event_date_formated'] >= current_state_date]
dates_to_dave_file = df_new['event_date_formated'].min(), df_new['event_date_formated'].max()

file_name = f"TODOIST-{dates_to_dave_file[0]}-{dates_to_dave_file[1]}.csv"

print(file_name)

df_new = df_new.drop('event_date_formated', axis=1)
#Salva novo df


TODOIST-20220414-20220419.csv


In [10]:
df_new.to_csv(os.path.join(path_data,file_name), index=False)